## prepare configs

In [19]:
!cd .. && python src/pipelines/prepare_configs.py --config config/pipeline_config.yml

## prepare dataset

In [20]:
!cd .. && python src/pipelines/prepare_dataset.py --config=experiments/prepare_dataset_config.yml

unzip archive...


create directories...


split train images by classes...


create validation dataset...


create test dataset...


create sample dataset...


## train model

In [21]:
!cd .. && python src/pipelines/train_model.py --config=experiments/train_model_config.yml

DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/vision-reproml, universal_newlines=False, shell=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/vision-reproml, universal_newlines=False, shell=None)
Epoch 0/2
----------
train Loss: 0.8615 Acc: 0.5000
val Loss: 0.7386 Acc: 0.5000
New best model found!
New record loss: 0.7386482357978821, previous record loss: inf

Epoch 1/2
----------
train Loss: 0.7168 Acc: 0.5000
val Loss: 0.6668 Acc: 0.5750
New best model found!
New record loss: 0.666775643825531, previous record loss: 0.7386482357978821

Epoch 2/2
----------
train Loss: 0.6534 Acc: 0.5450
val Loss: 0.6522 Acc: 0.5250
New best model found!
New record loss: 0.6521909236907959, previous record loss: 0.666775643825531

Training complete in 3m 6s
Best val Acc: 0.5250 Best val loss: 0.6522
INFO:root:data/sample image folder


## evaluate model

In [22]:
!cd .. && python src/pipelines/evaluate_model.py --config=experiments/evaluate_model_config.yml

100%|█████████████████████████████████████████████| 1/1 [00:11<00:00, 11.25s/it]


## open mlflow ui

### http://127.0.0.1:1234